# AES FPGA - testing

## Overlay
Requires .bit and .tcl files.

Syntax:
```python
from pynq.overlays.base import BaseOverlay
overlay = BaseOverlay("base.bit")
```

In [ ]:
from pynq import Overlay
from pynq import MMIO
import time

In [ ]:
# Dowload bit file to PYNQ
bit_file_name = "design_1_wrapper.bit"
base = Overlay(bit_file_name)

base.download()
base.is_loaded()

## `mmap` interface

### The Alma interface layer automatically detects the TTA and exposes the address map through status and control registers. Simply pass dummy value to read them.

In [ ]:
# mmap config
init_base_address = 0x43C00000
init_ctrl_size = 2**10    # 1kb control register memory
init_dmem_size = 2**16    # 16-bits
init_imem_size = 2**10    # 10-bits
init_pmem_size = 4*2**10  # 4kb paramter memory

region_size = max([init_ctrl_size, init_dmem_size, init_imem_size, init_pmem_size])

ctrl_mmap = MMIO(init_base_address, init_ctrl_size)
imem_mmap = MMIO(init_base_address + region_size, init_imem_size)
dmem_mmap = MMIO(init_base_address + (region_size*2), init_dmem_size)
pmem_mmap = MMIO(init_base_address + (region_size*3), init_pmem_size)

# print control memory segments
print ("DEVICECLASS : " + str(ctrl_mmap.read(0x300)))
print ("DEVICE_ID : " + str(ctrl_mmap.read(0x304)))
print ("INTERFACE_TYPE : " + str(ctrl_mmap.read(0x308)))
print ("CORE_COUNT : " + str(ctrl_mmap.read(0x30C)))
print ("CTRL_SIZE : " + str(ctrl_mmap.read(0x310)))
print ("DMEM_SIZE : " + str(ctrl_mmap.read(0x314)))
print ("IMEM_SIZE : " + str(ctrl_mmap.read(0x318)))
print ("PMEM_SIZE : " + str(ctrl_mmap.read(0x31C)))

new_ctrl_size = ctrl_mmap.read(0x310)
new_dmem_size = ctrl_mmap.read(0x314)
new_imem_size = ctrl_mmap.read(0x318)
new_pmem_size = ctrl_mmap.read(0x31C)

### Use the read value as memory map

In [ ]:
# mmap config based on the TTA core's value
base_address = init_base_address
ctrl_size = new_ctrl_size
dmem_size = new_dmem_size
imem_size = new_imem_size
pmem_size = new_pmem_size     # Address for second data memory

region_size = max([ctrl_size, dmem_size, imem_size, pmem_size])

ctrl_mmap = MMIO(base_address, ctrl_size)
imem_mmap = MMIO(base_address + region_size, imem_size)
dmem_mmap = MMIO(base_address + (region_size*2), dmem_size)
pmem_mmap = MMIO(base_address + (region_size*3), pmem_size)

# print control memory segments
print ("DEVICECLASS : " + str(ctrl_mmap.read(0x300)))
print ("DEVICE_ID : " + str(ctrl_mmap.read(0x304)))
print ("INTERFACE_TYPE : " + str(ctrl_mmap.read(0x308)))
print ("CORE_COUNT : " + str(ctrl_mmap.read(0x30C)))
print ("CTRL_SIZE : " + str(ctrl_mmap.read(0x310)))
print ("DMEM_SIZE : " + str(ctrl_mmap.read(0x314)))
print ("IMEM_SIZE : " + str(ctrl_mmap.read(0x318)))
print ("PMEM_SIZE : " + str(ctrl_mmap.read(0x31C)))
print ("DEBUG_FEATURE : " + str(ctrl_mmap.read(0x320)))
print ("BREAKPOINT_COUNT : " + str(ctrl_mmap.read(0x320)))

## Read data to memory

In [ ]:
# Write to instruction memory
imem_img = "lenet.img"
i=0
with open (imem_img, mode='r') as file:
    for line in file:
        line = line.strip().rjust(64,'0')
        for j in [1, 0]:
            i_word = int(line[(32*j):(32*j+32)], 2)
            imem_mmap.write(i, i_word)
            i += 4

In [ ]:
# Write to data memory
dmem_img = "lenet_data.img"
i=0
with open (dmem_img, mode='r') as file:
    for line in file:
        line = line.strip().rjust(32,'0')
        i_word = int(line, 2)
        dmem_mmap.write(i, i_word)
        i += 4

In [ ]:
# Trigger exeuction (Writing 0x2 to address 0x200 triggers TTA core)
ctrl_mmap.write(0x200, 0x2)

In [ ]:
time.sleep(5) # sleep for some time

In [ ]:
# Find physicall address for the ECC and LCC counter in simulator and set the address to following variables
#   you can get the address with "symbol_address predicted_number", 
#   "symbol_address ecc" and "symbol_address lcc" command in the proxim
predicted_number_addr = 8
ecc_addr = 12
lcc_addr = 16

In [ ]:
predicted_number = dmem_mmap.read(predicted_number_addr)
ecc_count = dmem_mmap.read(ecc_addr)
lcc_count = dmem_mmap.read(lcc_addr)
print("Predicted Number: " + str(predicted_number))
print("Execution cycle count: " + str(ecc_count))
print("Lock cycle count: " + str(ecc_count))